### Preparing assembly files for binning
To create a depth processing file, reads must be re-aligned to the contigs. This has been done using bowtie2 (can also be done using BWA). The next step would be to create a depth file with MetaBat2, convert that to be suitable for CONCOCT and MaxBin2, and then process these into bins. 

In [1]:
#this creates a depth file for MetaBat
#check names of the mapping files
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past
do
jgi_summarize_bam_contig_depths --outputDepth ../data/working/metabat_depth_"$f".txt ../../04_mapping/data/results/"$f".bam
done

SyntaxError: invalid syntax (<ipython-input-1-22de35fddd80>, line 3)

In [2]:
#this creates the MaxBin depth file SEE IF THERE IS AN ALTERNATIVE
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past; do
    jgi_summarize_bam_contig_depths --outputDepth ../data/working/mb2_master_depth_"$f".txt --noIntraDepthVariance ../../04_mapping/data/results/"$f".bam
    A=($(head -n 1 ../data/working/mb2_master_depth_"$f".txt)) 
    N=${#A[*]}
done
#the next bit breaks the depth file into different files for MaxBin to use       
for i in $(seq 4 $N); do 
		sample=$(head -n 1 ../data/working/mb2_master_depth_"$f".txt | cut -f $i) #dont think this line works yet
		grep -v totalAvgDepth ${out}/work_files/mb2_master_depth.txt | cut -f 1,$i > ${out}/work_files/mb2_${sample%.*}.txt
        

SyntaxError: invalid syntax (<ipython-input-2-70357cb867cc>, line 2)

In [ ]:
#this creates the CONCOCT depth file

#this part cuts up the contigs into 10kn files ARE THESE SCRIPTS CONCOCT INHERENT?
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past
		cut_up_fasta.py $../../02_assembly/"$f"_assembly.fa -c 10000 --merge_last -b ${out}/work_files/assembly_10K.bed -o 0 > ${out}/work_files/assembly_10K.fa

		comm "estimating contig fragment coverage..."	
		CMD="concoct_coverage_table.py ${out}/work_files/assembly_10K.bed ${out}/work_files/*.bam > ${out}/work_files/concoct_depth.txt"
		$(eval $CMD)
		if [[ $? -ne 0 ]]; then error "Something went wrong with estimating fragment abundance. Exiting..."; fi
	else
		comm "looks like contig coverage was already estimated... skipping"